<a href="https://colab.research.google.com/github/lautalom/diploUNC/blob/main/date_key_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# [Desafío](https://ciberseguridad.diplomatura.unc.edu.ar/cripto/doc/timerand.html)
El desafío consiste en descifrar un mensaje cifrado con una clave generada a partir de la fecha y hora de creación del mensaje.

Para obtener el mensaje a descifrar se debe hacer un requerimiento GET a una URL de la forma:

https://ciberseguridad.diplomatura.unc.edu.ar/cripto/timerand/<email\>/challenge
donde <email> debe ser reemplazado por una dirección de correo electrónico registrada.

El resultado será similar al siguiente:

From: miguel.montes@unc.edu.ar
Date: Sat Jan 29 19:15:45 UTC 2022
To: user@example.com

cJLPK7rt3EegpxZre4e7Sw2kncbwo2FuPjyiVn/cOLZ5wvZ3znoudW
YVGrVuWxP95Uc9aib0dUNMSBy+bJrdzFBqwb9M/6c9Z/7sy8RRcBOM
QH+5qAstqLqOAHeCwEFEiYUJt6q3++3RvHjRGkgA2Xu2LBdjr/B3sX
DmaFNF2QohmgzmrkyTxC3lH8rSN314I0qgDQWcDr0vSerrKWBFIMXh
0aezn9f+OHNjYOHBtCG0Pw/nnQLrjmFWLBo1AQ0we4oObutsWMak1O
+74GvBpOYLeNvIh8hL+uEEuFixxF8aTMUTsnZ13WxeZI2LH5ySf6QV
SFUDRWIKWryyv46KQZEFLidhoPR02blDMlty21TQc9spcl4Y2V5rKj
jsRosPzXDJr2qSSVoL7PVPoMB7WiS1rwzhgcPxFVk2aEF9YsMD1lfF
6azSNhEIfu028QYnuONG/Xof9Ni2+2coODD8CXSh7djCtw5Jex5HNu
RMY/9qWfk7u4RdR11kZhhUGfVH4r4Z5h5pAJHj8khZxlZ9AyFapO6F
Q6erUDIc4WEq6ns=

En el encabezado, además del emisor y el destinatario, aparece la hora de creación del mensaje. El cuerpo contiene una secuencia de bytes codificada en base64.

- Los primeros 128 bytes de esa cadena contienen una clave simétrica que ha sido cifrada con RSA de 1024 bits.
- El resto de la cadena, es el mensaje cifrado con AES-128 en modo CBC, utilizando relleno PKCS7, y con el IV colocado como primer bloque (16 bytes)
- La clave simétrica ha sido generada mediante la aplicación del algoritmo MD5 a la hora de creación del mensaje, expresada como el tiempo Unix expresado en microsegundos (es decir, la cantidad de microsegundos segundos transcurridos desde las cero horas del 1 de enero de 1970 UTC). **Como MD5 procesa secuencias de bytes, el número de microsegundos ha sido convertido en 8 bytes, utilizando la convención big endian.**

El mensaje está compuesto únicamente por caracteres ASCII.

Es necesario observar que la fecha y hora que figuran en el mensaje se encuentran expresadas con una precisión de segundos, no de microsegundos, por lo que existen 1.000.000 de claves posibles asociadas con ella.

In [ ]:
import requests
import base64
import struct
from datetime import datetime, timezone, timedelta
from hashlib import md5
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import hashes, padding as symmetric_padding
from cryptography.hazmat.primitives.asymmetric import padding as asymmetric_padding
from cryptography.hazmat.backends import default_backend

In [ ]:
email = "lautarolombardi19@gmail.com"
server = f"https://ciberseguridad.diplomatura.unc.edu.ar/cripto/timerand/{email}/challenge"

message = requests.get(url=server)
lines = message.text.split('\n')
# print(lines)
to_index = lines.index(f'To: {email}')
# Extract the body of the message
body = '\n'.join(lines[to_index + 2:])
date_line = next(line for line in lines if line.startswith('Date:'))
date_string = date_line.split(':', 1)[1].strip()
date_string

'Thu May 12 05:51:18 UTC 2022'

In [ ]:
decoded_body = base64.b64decode(body)
rsa_1024_key_hex = decoded_body[:128]
hex_iv = decoded_body[128:128+16].hex()
iv = bytes.fromhex(hex_iv)
msg = decoded_body[128+16:]
input_format = "%a %b %d %H:%M:%S %Z %Y"
date_object = datetime.strptime(date_string, input_format)

# Calculate Unix timestamp
unix_timestamp = int(date_object.replace(tzinfo=timezone.utc).timestamp())
microseconds = unix_timestamp * 1000000

print("Unix Time in Microseconds:", microseconds)

Unix Time in Microseconds: 1652334678000000


In [ ]:
microseconds_i = microseconds
for i in range(2**20):
  microbytes = microseconds_i.to_bytes(8, byteorder='big')
  possible_key = md5(microbytes).digest()
  cipher = Cipher(algorithms.AES(possible_key), modes.CBC(iv), backend=default_backend())
  decryptor = cipher.decryptor()
  try:
    decrypted_data = decryptor.update(msg) + decryptor.finalize()
    unpadder = symmetric_padding.PKCS7(128).unpadder()
    unpadded_data = unpadder.update(decrypted_data)
    unpadded_data += unpadder.finalize()
    decoded_text = unpadded_data.decode('ascii')
    print("Decrypted Data:", decoded_text)
    break
  except UnicodeDecodeError:
    pass
  except ValueError:
    pass
  finally:
    microseconds_i = microseconds + i


Decrypted Data: Making files is easy under the UNIX operating system.  Therefore, users
tend to create numerous files using large amounts of file space.  It has
been said that the only standard thing about all UNIX systems is the
message-of-the-day telling users to clean up their files.
		-- System V.2 administrator's guide


In [ ]:
answer_url = f"https://ciberseguridad.diplomatura.unc.edu.ar/cripto/timerand/{email}/answer"
response = requests.post(url=answer_url, files={"message": decoded_text})
print(response.text)


¡Ganaste!

